<a href="https://colab.research.google.com/github/josejustinos/projeto_alura/blob/main/projeto_alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import fitz  # PyMuPDF

def extrair_texto_pdf(caminho_arquivo):
    texto = ''
    with fitz.open(caminho_arquivo) as pdf:
        for pagina in pdf:
            texto += pagina.get_text()
    return texto

def encontrar_informacoes_concurso(texto):
    frases_de_busca = [
        "inscrição para concurso público",
        "vagas para concurso público"
    ]

    if any(frase in texto for frase in frases_de_busca):
        vagas = re.findall("vagas disponíveis: (\d+)", texto, re.IGNORECASE)
        escolaridade = re.findall("grau de escolaridade: (.+?)\n", texto, re.IGNORECASE)
        datas_inscricao = re.findall("inscrições: (\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})", texto, re.IGNORECASE)
        valor_inscricao = re.findall("valor da inscrição: (R\$[\d\.,]+)", texto, re.IGNORECASE)
        local_inscricao = re.findall("inscreva-se em: (.+?)\n", texto, re.IGNORECASE)

        return {
            "vagas": vagas,
            "escolaridade": escolaridade,
            "datas_inscricao": datas_inscricao,
            "valor_inscricao": valor_inscricao,
            "local_inscricao": local_inscricao
        }
    else:
        return None

def exibir_e_processar_informacoes(informacoes):
    if informacoes:
        print("Informações encontradas:")
        for chave, valor in informacoes.items():
            print(f"{chave.capitalize()}: {valor}")
    else:
        print("Não foram encontradas vagas de concurso público no documento.")

def main():
    while True:
        caminho_pdf = input("Por favor, insira o caminho do arquivo PDF: ")
        texto_pdf = extrair_texto_pdf(caminho_pdf)
        informacoes = encontrar_informacoes_concurso(texto_pdf)
        exibir_e_processar_informacoes(informacoes)

        escolha = input("Deseja analisar outro arquivo? Digite 's' para sim ou qualquer tecla para sair: ")
        if escolha.lower() != 's':
            break

if __name__ == "__main__":
    main()